In [27]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [10]:
predictor_small = Predictor.from_path("ser_dir_original_model/model.tar.gz")
predictor_large = Predictor.from_path("ser_dir_large_model/model.tar.gz")
predictor_orig = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/event2mind-2018.10.26.tar.gz")
predictor_mid = Predictor.from_path("ser_dir_5e_model/model.tar.gz")
predictor_xe = Predictor.from_path("xe_5e_model/model.tar.gz")
predictor_oe = Predictor.from_path("oe_5e_model/model.tar.gz")
predictor_xel = Predictor.from_path("xe_10e_model/model.tar.gz")
predictor_oel = Predictor.from_path("oe_10e_model/model.tar.gz")

to new params <allennlp.common.params.Params object at 0x7f1bb1962978>.
to new params <allennlp.common.params.Params object at 0x7f1bb1a16630>.


In [17]:

predictor_new_xi_2e = Predictor.from_path("xi_2e_new_model/model.tar.gz")
predictor_new_xi_10e = Predictor.from_path("xi_10e_new_model/model.tar.gz")

to new params <allennlp.common.params.Params object at 0x7f1bb57380f0>.
to new params <allennlp.common.params.Params object at 0x7f1bb523f518>.


In [7]:
predictor_birnn300_xi_2e = Predictor.from_path("xi_2e_birnn300_model/model.tar.gz")
predictor_birnn300_xi_10e = Predictor.from_path("xi_10e_birnn300_model/model.tar.gz")

In [10]:
predictor_cnn_xi_2e = Predictor.from_path("xi_2e_cnn_model/model.tar.gz")

In [2]:
predictor_alyona = Predictor.from_path("Alyona_model.tar.gz")

Преобразуем данные из test.csv, парсим по колонкам

In [3]:
test_df = pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

Группируем по фразам отдельно намерения, эмоции субъекта и эмоции окружающих

UPD: Убираем из строк не несущие смысла to, to be, is и точки.

In [4]:
def clear_word(w):
    x = w
    if x[:3] == 'is ' or x[:3] == 'to ':
        x = x[3:]
    if x[:3] == 'be ':
        x = x[3:]
    if x[len(x) - 1] == '.':\
        x = x[:len(x)-1]
    return x

In [5]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []
set_xi = set()
set_xe = set()
set_oe = set()

phrase_set = set([x for x in test_df.Event])

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set()
    for x in set(res_xi):
        if x != '':
            phrase2xintent[phrase].add(clear_word(x))
    res_xi.clear()
    
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set()
    for x in set(res_xe):
        if x != '':
            phrase2xemotion[phrase].add(clear_word(x))
    res_xe.clear()
    
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set()
    for x in set(res_oe):
        if x != '':
            phrase2oemotion[phrase].add(clear_word(x))
    res_oe.clear()

Считаем recall следующим образом: из всех вариантов, выданных моделью для данной фразы, оставляем те, что есть в контрольном множестве фразы, сформированном ранее, и делим их количество на размер контрольного множества. Затем делим полученную сумму на количество фраз. Проделываем для каждого из выходов (intent, xreact и oreact)

In [6]:
def count_recall(predictor):
    res_oe = 0
    res_xe = 0
    res_xi = 0

    for phrase in phrase_set:
        result = predictor.predict(source=phrase)

        oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
        intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
        xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

        res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
        res_xe += len(phrase2xemotion[phrase].intersection(xreact)) / len(phrase2xemotion[phrase])
        res_xi += len(phrase2xintent[phrase].intersection(intents)) / len(phrase2xintent[phrase])

    print("Recall for intents: ", res_xi / len(phrase_set))
    print("Recall for x_react: ", res_xe / len(phrase_set))
    print("Recall for o_react: ", res_oe / len(phrase_set))
    print()
    
def count_recall_all():    
    print("Model with xintent metric:")
    print()
    print("Recall of model trained for 10 epochs")
    count_recall(predictor_large)
    print("Recall of model trained for 5 epochs")
    count_recall(predictor_mid)
    print("Recall of model trained for 2 epochs")
    count_recall(predictor_small)
    
    print("Model with xreact metric:")
    print()
    print("Recall of model trained for 10 epochs")
    count_recall(predictor_xel)
    print("Recall of model trained for 5 epochs")
    count_recall(predictor_xe)
    
    print("Model with oreact metric:")
    print()
    print("Recall of model trained for 10 epochs")
    count_recall(predictor_oel)
    print("Recall of model trained for 5 epochs")
    count_recall(predictor_oe)
    
    print("Recall of original model")
    count_recall(predictor_orig)
    

In [9]:
count_recall_all()

Model with xintent metric:

Recall of model trained for 10 epochs
Recall for intents:  0.2909114863298134
Recall for x_react:  0.1786399376040804
Recall for o_react:  0.6410126689011149

Recall of model trained for 5 epochs
Recall for intents:  0.2824079343999664
Recall for x_react:  0.19815658002908923
Recall for o_react:  0.6493012078669447

Recall of model trained for 2 epochs
Recall for intents:  0.24537089736292872
Recall for x_react:  0.2539724699087242
Recall for o_react:  0.6486898964986615

Model with xreact metric:

Recall of model trained for 10 epochs
Recall for intents:  0.2616591833723305
Recall for x_react:  0.2460222601656857
Recall for o_react:  0.6493855266763631

Recall of model trained for 5 epochs
Recall for intents:  0.2616591833723305
Recall for x_react:  0.2460222601656857
Recall for o_react:  0.6493855266763631

Model with oreact metric:

Recall of model trained for 10 epochs
Recall for intents:  0.24537089736292872
Recall for x_react:  0.2539724699087242
Recal

In [16]:
count_recall(predictor_new_xi_2e)

Recall for intents:  0.23960876072429815
Recall for x_react:  0.2895918969624146
Recall for o_react:  0.6595754547945781



In [18]:
count_recall(predictor_new_xi_10e)

Recall for intents:  0.28993338814055963
Recall for x_react:  0.13174813971626703
Recall for o_react:  0.6015219545100017



In [20]:
count_recall(predictor_birnn300_xi_2e)

Recall for intents:  0.28294546681000865
Recall for x_react:  0.2503920824637948
Recall for o_react:  0.6500137018065305



In [12]:
count_recall(predictor_birnn300_xi_10e)

Recall for intents:  0.30106347098379005
Recall for x_react:  0.20521722633276354
Recall for o_react:  0.6448997660153045



In [11]:
count_recall(predictor_cnn_xi_2e)

Recall for intents:  0.2789202976453973
Recall for x_react:  0.24991462720546334
Recall for o_react:  0.6075444254727127



In [7]:
count_recall(predictor_alyona)

Recall for intents:  0.19982714644069197
Recall for x_react:  0.0
Recall for o_react:  0.5354507894348536



In [24]:
def do_friends(predictor):
    result_file = open("perfect_sent_res.txt", "w")
    with open("perfect_sent.txt", "r") as file:
        for phrase in file:
            result = predictor.predict(source=phrase)
            phrase = phrase[:len(phrase)-1]

            oreact = [" ".join(react) for react in result["oreact_top_k_predicted_tokens"]]    
            intents = [" ".join(react) for react in result["xintent_top_k_predicted_tokens"]]
            xreact = [" ".join(react) for react in result["xreact_top_k_predicted_tokens"]]
            size = min(len(oreact), len(xreact), len(intents))
            lines = []
            for i in range(size):
                lines.append(phrase + ',' + intents[i] + ',' + xreact[i] + ',' + oreact[i] + '\n')

            for line in lines:
                result_file.write(line)
        
    result_file.close()

In [26]:
do_friends(predictor_alyona)

In [37]:
def do_beauty(setty):
    res_str = ""
    for line in setty:
        if line.find("@@UNKNOWN@@") == -1 and line.find("none") == -1:
            res_str = res_str + line + ', '
    return res_str[:len(int_str)-2]

In [40]:
phrase = "Наша команда благорадит вас за внимание"
result = predictor_alyona.predict(source="Наша команда благорадит вас за внимание")

oreact = do_beauty([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])  
intents = do_beauty([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
xreact = do_beauty([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

print(phrase, "\n")
print("Intent: ", intents)
print("Xreact: ", xreact)
print("Oreact: ", oreact)

Наша команда благорадит вас за внимание 

Intent:  учить, быть успешным, прятаться, помогать другим, быть хорошим
Xreact:  счастливый, хорошо, довольный, полезный, гордый, образованный,
Oreact:  счастливый, благодарный, заинтересованный, довольный, хорошо, 
